# Housing Recommendation based on distance to nearby utilities

**Authored by**:  Linh Huong Nguyen

**Duration**: 90 mins

**Level**: Intermediate

**Pre-requisite Skills**: Python, Pandas, Matplotlib, NumbPy, Seaborn, Scikit-learn


### Scenario

As a tenant looking for rental houses, I want to calculate the distance between rental listings and key public amenities such as public transportation hubs, schools, and public landmarks in the City of Melbourne, so that I can choose the most suitable place to apply for rent.

### What this use case will teach you

At the end of this use case, you will have demonstrated the following skills:

* Accessed and imported geospatial and rental listing datasets from open data portals and APIs.

* Performed data cleaning, preprocessing, and geocoding of addresses to ensure spatial accuracy.

* Used geospatial libraries to calculate distances between points of interest (POIs) and rental properties.

* Conducted exploratory data analysis (EDA) to assess accessibility patterns and disparities.

* Visualized geospatial data on interactive maps to highlight proximity patterns and coverage gaps.

* Derived actionable insights to inform housing accessibility and urban development policies.

### Background and Introduction

The accessibility of essential public amenities such as transportation hubs, schools, and public landmarks plays a significant role in shaping rental market dynamics, urban livability, and resident satisfaction. For renters, proximity to these amenities can influence housing decisions, commute times, and quality of life. For policymakers and urban planners, understanding how well rental properties are served by these facilities is crucial for identifying underserved areas, prioritizing infrastructure investments, and ensuring equitable access across the community.

This use case addresses the need for a data-driven approach to evaluating the spatial relationship between rental housing and public amenities in the City of Melbourne. By combining rental property data with geospatial datasets of public transport stations, schools, and notable landmarks, we can calculate precise distances and analyze accessibility patterns across the city. These insights can help guide housing policy, transport planning, and community development initiatives.

### Datasets used


* City of Melbourne Public Transport Stops
This dataset contains the locations of public transport stops (including train, tram, and bus) within the City of Melbourne. It includes stop names, modes, geographic coordinates, and route information. Data is sourced from the Melbourne Open Data portal and accessible via API V2.1.

* Victoria School Locations 2024
This dataset contains the list of all school locations in Victoria, including primary and secondary schools, government and non-government. Attributes include school name, sector, type, address, phone, and geographic coordinates. Data is sourced from the Victorian Department of Education and accessible via API V2.1.

* City of Melbourne Landmarks and Places of Interest
This dataset contains key public landmarks, cultural sites, and recreational facilities within the City of Melbourne. It includes location names, categories, and geographic coordinates. Data is sourced from the Melbourne Open Data portal and accessible via API V2.1.

* [Rental Listings Dataset]
This dataset contains current rental property listings in the City of Melbourne. Attributes may include address, rental price, property type, number of bedrooms, and listing date. Addresses will be geocoded for spatial analysis.

### Importing Datasets

This section imports essential libraries for data manipulation, visualization, geospatial analysis, interactive mapping, and fetching data from APIs. These libraries provide the necessary functionality for processing, analyzing, and visualizing the project data effectively.

In [123]:
from geopy.geocoders import Nominatim
from scipy.spatial import KDTree
from geopy.distance import geodesic
import openrouteservice
import requests
import pandas as pd
import os
from io import StringIO
import requests
import seaborn as sns
import folium
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import shape, Point
import json
import zipfile
from io import BytesIO
import re


### Loading the datasets using API 2.1v

This section defines functions for fetching data from APIs. The API_Unlimited function retrieves datasets from the Melbourne Open Data Portal using dataset IDs, processes the data into a DataFrame, and provides a preview for verification. Similarly, the fetch_data_from_url function fetches data directly from a given URL, processes it into a DataFrame, and displays a sample for validation. These functions enable seamless access to external datasets for analysis.

In [124]:
def API_Unlimited(datasetname): # pass in dataset name and api key
    dataset_id = datasetname

    base_url = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'
    #apikey = api_key
    dataset_id = dataset_id
    format = 'csv'

    url = f'{base_url}{dataset_id}/exports/{format}'
    
    params = {
        'select': '*',
        'limit': -1,  # all records
        'lang': 'en',
        'timezone': 'UTC'
    }

    # GET request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        # StringIO to read the CSV data
        url_content = response.content.decode('utf-8')
        datasetname = pd.read_csv(StringIO(url_content), delimiter=';')
        print(datasetname.sample(10, random_state=999)) # Test
        return datasetname 
    else:
        return (print(f'Request failed with status code {response.status_code}'))

In [125]:
def API_Unlimited_external(datasetname): # pass in dataset name and api key
    dataset_id = datasetname

    base_url = 'https://www.education.vic.gov.au/Documents/about/research/datavic/'
    dataset_id = dataset_id
    format = 'csv'

    url = f'{base_url}{dataset_id}.{format}'
    params = {
        'select': '*',
        'limit': -1,  # all records
        'lang': 'en',
        'timezone': 'UTC'
    }

    # GET request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        # StringIO to read the CSV data
        url_content = response.text
        datasetname = pd.read_csv(StringIO(url_content), delimiter=',')
        print(datasetname.sample(10, random_state=999)) # Test
        return datasetname 
    else:
        return (print(f'Request failed with status code {response.status_code}'))



### Fetching and Previewing Datasets

This section defines the dataset download links required for the use case and fetches the corresponding data using the API_Unlimited function. The datasets include school locations and landscapes which are essential for calculating distance from rental listings to utilities.

In [126]:
download_link_1 = 'landmarks-and-places-of-interest-including-schools-theatres-health-services-spor'
download_link_2 = 'dv378_DataVic-SchoolLocations-2024'

# Use functions to download and load data
landmarks = API_Unlimited(download_link_1)
school_locations = API_Unlimited_external(download_link_2)

                  theme                                        sub_theme  \
30   Leisure/Recreation               Major Sports & Recreation Facility   
18   Leisure/Recreation  Informal Outdoor Facility (Park/Garden/Reserve)   
154           Transport                               Transport Terminal   
73            Transport                                  Railway Station   
20            Transport                                  Railway Station   
195           Transport                                  Railway Station   
165              Office                                           Office   
125  Leisure/Recreation               Major Sports & Recreation Facility   
85        Community Use                                 Public Buildings   
54   Leisure/Recreation                     Private Sports Club/Facility   

                           feature_name                         co_ordinates  
30       Melbourne Cricket Ground (MCG)  -37.8194921618419, 144.983402879078  
18   

In [127]:
school_locations= school_locations.dropna(subset=['Y', 'X'])

In [128]:
landmarks.head()

,theme,sub_theme,feature_name,co_ordinates
0,Community Use,Fire Station,Metropolitan Fire Brigade (MFB),"-37.8092318636838, 144.975247619376"
1,Place Of Assembly,Art Gallery/Museum,Koorie Heritage Trust Inc,"-37.8133854259085, 144.954027907736"
2,Education Centre,Tertiary (University),RMIT University,"-37.8080795360545, 144.964452974798"
3,Leisure/Recreation,Informal Outdoor Facility (Park/Garden/Reserve),Macarthur Square,"-37.7983318676737, 144.971514146104"
4,Transport,Railway Station,North Melbourne Railway Station,"-37.8073823625058, 144.942429848025"


In [129]:
landmarks_clean=landmarks.dropna(subset='co_ordinates')
landmarks_clean[['latitude', 'longitude']] =landmarks_clean['co_ordinates'].str.split(',', expand=True).astype(float)
landmarks_clean=landmarks_clean.drop(columns=['co_ordinates'])
landmarks_clean.head()

,theme,sub_theme,feature_name,latitude,longitude
0,Community Use,Fire Station,Metropolitan Fire Brigade (MFB),-37.809232,144.975248
1,Place Of Assembly,Art Gallery/Museum,Koorie Heritage Trust Inc,-37.813385,144.954028
2,Education Centre,Tertiary (University),RMIT University,-37.808080,144.964453
3,Leisure/Recreation,Informal Outdoor Facility (Park/Garden/Reserve),Macarthur Square,-37.798332,144.971514
4,Transport,Railway Station,North Melbourne Railway Station,-37.807382,144.942430


### GTFS Schedule Dataset

The GTFS Schedule dataset contains static timetable information of public transport services in Victoria

In [130]:

current_directory = os.getcwd()
dataset_folder = 'mpt_data'
dataset_path = os.path.join(current_directory, dataset_folder)
inner_zip_paths = ['2/google_transit.zip', '3/google_transit.zip', '4/google_transit.zip']

In [131]:
def API_GTFS(url: str, inner_zip_paths: list) -> dict:

    required_files = ['stops.txt', 'stop_times.txt', 'routes.txt', 'trips.txt', 'calendar.txt']
    datasets = {}
    # Download main zip
    response = requests.get(url)
    response.raise_for_status()

    # Open main zip in memory
    with zipfile.ZipFile(BytesIO(response.content)) as main_zip:
        for inner_zip_path in inner_zip_paths:
            if inner_zip_path not in main_zip.namelist():
                continue

            subfolder_name = os.path.basename(os.path.dirname(inner_zip_path))
            datasets[subfolder_name] = {}

            with main_zip.open(inner_zip_path) as inner_zip_file:
                with zipfile.ZipFile(BytesIO(inner_zip_file.read())) as inner_zip:
                    for file_name in required_files:
                        if file_name in inner_zip.namelist():
                            with inner_zip.open(file_name) as f:
                                datasets[subfolder_name][file_name] = pd.read_csv(f)

    return datasets


In [132]:
url = 'https://data.ptv.vic.gov.au/downloads/gtfs.zip'
inner_zip_paths = ['2/google_transit.zip', '3/google_transit.zip', '4/google_transit.zip']

datasets  = API_GTFS(url,inner_zip_paths)

C:\Users\phuon\AppData\Local\Temp\ipykernel_35476\2046210630.py:23: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  datasets[subfolder_name][file_name] = pd.read_csv(f)
C:\Users\phuon\AppData\Local\Temp\ipykernel_35476\2046210630.py:23: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  datasets[subfolder_name][file_name] = pd.read_csv(f)


In [133]:
train_stops=datasets["2"]["stops.txt"]
tram_stops=datasets["3"]["stops.txt"]
bus_stops=datasets["4"]["stops.txt"]

In [134]:
train_stops.head()

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,level_id,platform_code
0,10117,Jordanville Station,-37.873763,145.112473,NaN,vic:rail:JOR,1.0,Level 0,1
1,10920,Flagstaff Station,-37.811880,144.956043,NaN,vic:rail:FGS,1.0,Level -2,1
2,10921,Flagstaff Station,-37.811725,144.955968,NaN,vic:rail:FGS,1.0,Level -2,2
3,10922,Melbourne Central Station,-37.809974,144.962547,NaN,vic:rail:MCE,1.0,Level -2,1
4,10923,Melbourne Central Station,-37.809865,144.962505,NaN,vic:rail:MCE,1.0,Level -2,2


In [135]:
tram_stops.head()

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,level_id
0,10311,Glenferrie Rd/Wattletree Rd #45,-37.862455,145.028508,NaN,NaN,0,Level 0
1,10371,Duncraig Ave/Wattletree Rd #44,-37.862069,145.025382,NaN,NaN,0,Level 0
2,1083,Clyde St/Raleigh Rd #42,-37.769699,144.898841,NaN,NaN,0,Level 0
3,11285,Egerton Rd/Wattletree Rd #43,-37.861710,145.022754,NaN,NaN,0,Level 0
4,1185,Vincent St/Wattletree Rd #50,-37.864226,145.043375,NaN,NaN,0,Level 0


In [136]:
bus_stops.head()

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,level_id
0,1000,Dole Ave/Cheddar Rd,-37.700775,145.018951,NaN,NaN,0,Level 0
1,10001,Rex St/Taylors Rd,-37.726975,144.776152,NaN,NaN,0,Level 0
2,10002,Yuille St/Centenary Ave,-37.676159,144.595789,NaN,NaN,0,Level 0
3,10009,Gum Rd/Main Rd West,-37.741497,144.775899,NaN,NaN,0,Level 0
4,1001,Lloyd Ave/Cheddar Rd,-37.699183,145.019685,NaN,NaN,0,Level 0


### Rental Listings Spreadsheet

A spreadsheet of rental listings is collected from realestate.com.au to be ranked based on their distance to utilities

In [137]:
rental_listings = pd.read_excel("Rental Listings.xlsx")
rental_listings.head()

,Suburb,Postcode,Address,Price pw,Available date,Number of bedrooms,Number of bathrooms,Number of carspaces,Type of property
0,Carlton,3053,"2512/551 Swanston Street, Carlton, Vic",755,2025-08-25,2,2,0,Apartment
1,Carlton,3053,"2006/28 Bouverie Street, Carlton, Vic",795,2025-08-11,2,2,0,Apartment
2,Carlton,3053,"707A/640 Swanston Street, Carlton, Vic",700,2025-08-25,2,1,1,Apartment
3,Carlton,3053,"23/411 Lygon Street, Carlton, Vic",660,2025-08-05,2,1,1,Townhouse
4,Carlton,3053,"311/127 Cardigan Street, Carlton, Vic",850,2025-08-05,2,2,1,Apartment


In [138]:
rental_listings['Address'] = rental_listings['Address'].str.split("/").str[-1]
rental_listings.head()

,Suburb,Postcode,Address,Price pw,Available date,Number of bedrooms,Number of bathrooms,Number of carspaces,Type of property
0,Carlton,3053,"551 Swanston Street, Carlton, Vic",755,2025-08-25,2,2,0,Apartment
1,Carlton,3053,"28 Bouverie Street, Carlton, Vic",795,2025-08-11,2,2,0,Apartment
2,Carlton,3053,"640 Swanston Street, Carlton, Vic",700,2025-08-25,2,1,1,Apartment
3,Carlton,3053,"411 Lygon Street, Carlton, Vic",660,2025-08-05,2,1,1,Townhouse
4,Carlton,3053,"127 Cardigan Street, Carlton, Vic",850,2025-08-05,2,2,1,Apartment


### Generate latitude and longitude from address

The below code is a function to return the latitude and longitude from a passed in address.

In [143]:
#Function to geocode an address using Nominatim
def geocode_address(address):
    geolocator = Nominatim(user_agent="mapping_app1.0",timeout=15)
    
    #Define the bounding box for Melbourne
    melbourne_bbox = [(-38.5267, 144.5937), (-37.5113, 145.5125)] 
    
    #Geocode the address within the Melbourne bounding box
    location = geolocator.geocode(address, viewbox=melbourne_bbox, bounded=True)
    
    if location:
        return location.latitude, location.longitude
    else:
        return None
    


In [144]:
rental_listings['coords'] = rental_listings['Address'].apply(geocode_address)
rental_listings.head()

,Suburb,Postcode,Address,Price pw,Available date,Number of bedrooms,Number of bathrooms,Number of carspaces,Type of property,coords
0,Carlton,3053,"551 Swanston Street, Carlton, Vic",755,2025-08-25,2,2,0,Apartment,"(-37.8056782, 144.9623566)"
1,Carlton,3053,"28 Bouverie Street, Carlton, Vic",795,2025-08-11,2,2,0,Apartment,"(-37.8056215, 144.9618704)"
2,Carlton,3053,"640 Swanston Street, Carlton, Vic",700,2025-08-25,2,1,1,Apartment,"(-37.8048744, 144.9632684)"
3,Carlton,3053,"411 Lygon Street, Carlton, Vic",660,2025-08-05,2,1,1,Townhouse,"(-37.7948366, 144.9675549)"
4,Carlton,3053,"127 Cardigan Street, Carlton, Vic",850,2025-08-05,2,2,1,Apartment,"(-37.7976527, 144.9660059)"


In [162]:
rental_listings_clean=rental_listings.dropna(subset='coords')
rental_listings_clean[['latitude', 'longitude']] =pd.DataFrame(
    rental_listings_clean['coords'].tolist(), index=rental_listings_clean.index)
rental_listings_clean=rental_listings_clean.drop(columns=['coords'])
rental_listings_clean.head()

C:\Users\phuon\AppData\Local\Temp\ipykernel_35476\164360247.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rental_listings_clean[['latitude', 'longitude']] =pd.DataFrame(
C:\Users\phuon\AppData\Local\Temp\ipykernel_35476\164360247.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rental_listings_clean[['latitude', 'longitude']] =pd.DataFrame(


,Suburb,Postcode,Address,Price pw,Available date,Number of bedrooms,Number of bathrooms,Number of carspaces,Type of property,latitude,longitude
0,Carlton,3053,"551 Swanston Street, Carlton, Vic",755,2025-08-25,2,2,0,Apartment,-37.805678,144.962357
1,Carlton,3053,"28 Bouverie Street, Carlton, Vic",795,2025-08-11,2,2,0,Apartment,-37.805622,144.961870
2,Carlton,3053,"640 Swanston Street, Carlton, Vic",700,2025-08-25,2,1,1,Apartment,-37.804874,144.963268
3,Carlton,3053,"411 Lygon Street, Carlton, Vic",660,2025-08-05,2,1,1,Townhouse,-37.794837,144.967555
4,Carlton,3053,"127 Cardigan Street, Carlton, Vic",850,2025-08-05,2,2,1,Apartment,-37.797653,144.966006


From the above codes, rental listings table has been added longtitude and latitude columns to navigate the listing on a map.

In [163]:
# Average lat/lon for center
melbourne_center = [rental_listings_clean['latitude'].mean(), rental_listings_clean['longitude'].mean()]
listing_map = folium.Map(location=melbourne_center, zoom_start=13)

# Add coworking space markers
for _, row in rental_listings_clean.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['Address'],
    ).add_to(listing_map)


# Display the map
listing_map


All rental listings have been visualised on a map. By clicking on each marker, the rental listing's address is shown.

It is visible that the listings are within City of Melbourne, spread out to different suburbs.

### Find nearest station

The below code is a function to return the nearest station from each rental listing location, using a ML model called KDTree.

In [164]:
def find_nearest_station(lat, lon, kdtree, df):
    #Query the KDTree with the given latitude and longitude to find nearest station. Distance is returned in degrees so need to calculate the meters
    distance, index = kdtree.query([lat, lon])

    #Get the nearest station details from the DataFrame
    nearest_station = df.iloc[index]

    #Extract stations coords
    nearest_station_coords = (nearest_station["stop_lat"], nearest_station["stop_lon"])
    point_coords = (lat, lon)

    #Calculate the geodesic distance (in meters) between the point and the nearest statio
    distance_meters = geodesic(point_coords, nearest_station_coords).meters
    
    return nearest_station, distance_meters

Then the function is applied to the dataset which contains all coordinates of train stations.

In [165]:
train_stations_coords = train_stops[["stop_lat", "stop_lon"]].values
kdtree_train = KDTree(train_stations_coords)
rental_listings_clean[["nearest_station", "distance_meters_station"]] = (
    rental_listings_clean.apply(
        lambda r: pd.Series(
            find_nearest_station(r["latitude"], r["longitude"], kdtree_train, train_stops)
        ),
        axis=1
    )
)
rental_listings_clean.head()

,Suburb,Postcode,Address,Price pw,Available date,Number of bedrooms,Number of bathrooms,Number of carspaces,Type of property,latitude,longitude,nearest_station,distance_meters_station
0,Carlton,3053,"551 Swanston Street, Carlton, Vic",755,2025-08-25,2,2,0,Apartment,-37.805678,144.962357,stop_id 109...,464.903313
1,Carlton,3053,"28 Bouverie Street, Carlton, Vic",795,2025-08-11,2,2,0,Apartment,-37.805622,144.961870,stop_id 109...,474.314037
2,Carlton,3053,"640 Swanston Street, Carlton, Vic",700,2025-08-25,2,1,1,Apartment,-37.804874,144.963268,stop_id 109...,558.004086
3,Carlton,3053,"411 Lygon Street, Carlton, Vic",660,2025-08-05,2,1,1,Townhouse,-37.794837,144.967555,stop_id 109...,1726.334823
4,Carlton,3053,"127 Cardigan Street, Carlton, Vic",850,2025-08-05,2,2,1,Apartment,-37.797653,144.966006,stop_id 109...,1390.119407


From the above table, Nearest station column and its coordinate is added to the listing table. As a result, distance from each rental listing to its nearest station is also calculated and added to the table.

### Find nearest school

Similarly, a function is created to find the nearest school

In [167]:
def find_nearest_school(lat, lon, kdtree, df):
    #Query the KDTree with the given latitude and longitude to find nearest station. Distance is returned in degrees so need to calculate the meters
    distance, index = kdtree.query([lat, lon])

    #Get the nearest station details from the DataFrame
    nearest_school = df.iloc[index]

    #Extract stations coords
    nearest_school_coords = (nearest_school["Y"], nearest_school["X"])
    point_coords = (lat, lon)

    #Calculate the geodesic distance (in meters) between the point and the nearest statio
    distance_meters = geodesic(point_coords, nearest_school_coords).meters
    school_name = nearest_school.get("School_Name", None)
    
    return school_name, distance_meters

In [168]:
nearest_school_coords = school_locations[["Y", "X"]].values
kdtree_school = KDTree(nearest_school_coords)
rental_listings_clean[["nearest_school", "distance_meters_school"]] = (
    rental_listings_clean.apply(
        lambda r: pd.Series(
            find_nearest_school(r["latitude"], r["longitude"],kdtree_school, school_locations)
        ),
        axis=1
    )
)
rental_listings_clean.head()

,Suburb,Postcode,Address,Price pw,Available date,Number of bedrooms,Number of bathrooms,Number of carspaces,Type of property,latitude,longitude,nearest_station,distance_meters_station,nearest_school,distance_meters_school
0,Carlton,3053,"551 Swanston Street, Carlton, Vic",755,2025-08-25,2,2,0,Apartment,-37.805678,144.962357,stop_id 109...,464.903313,River Nile School,642.216005
1,Carlton,3053,"28 Bouverie Street, Carlton, Vic",795,2025-08-11,2,2,0,Apartment,-37.805622,144.961870,stop_id 109...,474.314037,River Nile School,599.027095
2,Carlton,3053,"640 Swanston Street, Carlton, Vic",700,2025-08-25,2,1,1,Apartment,-37.804874,144.963268,stop_id 109...,558.004086,Carlton Gardens Primary School,632.785299
3,Carlton,3053,"411 Lygon Street, Carlton, Vic",660,2025-08-05,2,1,1,Townhouse,-37.794837,144.967555,stop_id 109...,1726.334823,Carlton Primary School,268.698675
4,Carlton,3053,"127 Cardigan Street, Carlton, Vic",850,2025-08-05,2,2,1,Apartment,-37.797653,144.966006,stop_id 109...,1390.119407,Carlton Primary School,452.892642


From the above codes, nearest schools, coordinates and distances to rental listings are added to the table.

### Find nearest landmark

The function below finds the nearest landmark, its coordinate and distance to the rental listing.

In [170]:
def find_nearest_landmark(lat, lon, kdtree, df):
    #Query the KDTree with the given latitude and longitude to find nearest station. Distance is returned in degrees so need to calculate the meters
    distance, index = kdtree.query([lat, lon])

    #Get the nearest station details from the DataFrame
    nearest_landmark = df.iloc[index]

    #Extract stations coords
    nearest_landmark_coords = (nearest_landmark["latitude"], nearest_landmark["longitude"])
    point_coords = (lat, lon)

    #Calculate the geodesic distance (in meters) between the point and the nearest statio
    distance_meters = geodesic(point_coords, nearest_landmark_coords).meters
    landmark = nearest_landmark.get("feature_name", None)
    
    return landmark, distance_meters

In [171]:
nearest_landmark_coords = landmarks_clean[["latitude", "longitude"]].values
kdtree_landmark = KDTree(nearest_landmark_coords)
rental_listings_clean[["nearest_landmark", "distance_meters_landmark"]] = (
    rental_listings_clean.apply(
        lambda r: pd.Series(
            find_nearest_landmark(r["latitude"], r["longitude"],kdtree_landmark, landmarks_clean)
        ),
        axis=1
    )
)
rental_listings_clean.head()

,Suburb,Postcode,Address,Price pw,Available date,Number of bedrooms,Number of bathrooms,Number of carspaces,Type of property,latitude,longitude,nearest_station,distance_meters_station,nearest_school,distance_meters_school,nearest_landmark,distance_meters_landmark
0,Carlton,3053,"551 Swanston Street, Carlton, Vic",755,2025-08-25,2,2,0,Apartment,-37.805678,144.962357,stop_id 109...,464.903313,River Nile School,642.216005,City Baths,179.945130
1,Carlton,3053,"28 Bouverie Street, Carlton, Vic",795,2025-08-11,2,2,0,Apartment,-37.805622,144.961870,stop_id 109...,474.314037,River Nile School,599.027095,City Baths,206.407972
2,Carlton,3053,"640 Swanston Street, Carlton, Vic",700,2025-08-25,2,1,1,Apartment,-37.804874,144.963268,stop_id 109...,558.004086,Carlton Gardens Primary School,632.785299,Lincoln Square,235.417010
3,Carlton,3053,"411 Lygon Street, Carlton, Vic",660,2025-08-05,2,1,1,Townhouse,-37.794837,144.967555,stop_id 109...,1726.334823,Carlton Primary School,268.698675,All Nations Uniting Church,173.551007
4,Carlton,3053,"127 Cardigan Street, Carlton, Vic",850,2025-08-05,2,2,1,Apartment,-37.797653,144.966006,stop_id 109...,1390.119407,Carlton Primary School,452.892642,The Ian Potter Museum Of Art,165.380552


In the above table, all rental listings locations and their information of closest train station, school and landmark is shown.

### Ranking rental listings

In [172]:
rental_listings_ranking = rental_listings_clean.copy()


In [174]:
# Normalise columns:
cols = ["distance_meters_station", "distance_meters_school", "distance_meters_landmark"]
def minmax_normalize(s: pd.Series) -> pd.Series:
    s_min, s_max = s.min(), s.max()
    if pd.isna(s_min) or pd.isna(s_max) or s_max == s_min:
        # all values equal or empty -> return zeros
        return pd.Series(0.0, index=s.index)
    return (s - s_min) / (s_max - s_min)
# Normalize each distance (0 = closest, 1 = farthest)
for c in cols:
    rental_listings_ranking[f"norm_{c}"] = minmax_normalize(rental_listings_ranking[c])

# Equal-importance composite: average of normalized distances
rental_listings_ranking["accessibility_score"] = (
    rental_listings_ranking[[f"norm_{c}" for c in cols]].mean(axis=1)
)

# Rank: smaller score = closer overall (rank 1 is best)
rental_listings_ranking["accessibility_rank"] = (
    rental_listings_ranking["accessibility_score"].rank(method="dense", ascending=True).astype(int)
)

The above code normalises the distance columns, calculates an accessibility score by averaging among all distance matrices and lastly ranks the rental listings by their accessibility score.

In [176]:
top10_listing = (rental_listings_ranking
         .sort_values(["accessibility_rank", "accessibility_score"], ascending=[True, True])
         .head(10)
         .copy())
top10_listing

,Suburb,Postcode,Address,Price pw,Available date,Number of bedrooms,Number of bathrooms,Number of carspaces,Type of property,latitude,...,distance_meters_station,nearest_school,distance_meters_school,nearest_landmark,distance_meters_landmark,norm_distance_meters_station,norm_distance_meters_school,norm_distance_meters_landmark,accessibility_score,accessibility_rank
13,East Melbourne,3002,"16 Vale Street, East Melbourne, Vic",1500,2025-08-05,3,3,1,Townhouse,-37.819209,...,463.217654,Melbourne Indigenous Transition School,37.799155,Richmond Football Club,287.563630,0.077132,0.000000,0.005540,0.027557,1
78,St Kilda,3182,"6 Redan Street, St Kilda, Vic",565,2025-08-29,2,1,1,Apartment,-37.860181,...,459.178130,Cheder Levi Yitzchok Inc,47.311062,Wesley College,1539.034326,0.076364,0.005686,0.033246,0.038432,2
22,Melbourne,3000,"633 Little Lonsdale Street, Melbourne, Vic",800,2025-09-20,2,1,0,Apartment,-37.813811,...,301.006607,Ozford College,152.048738,Koorie Heritage Trust Inc,91.202579,0.046306,0.068298,0.001193,0.038599,3
18,Flemington,3031,"125D Victoria Street, Flemington, Vic",700,2025-09-03,3,1,0,House,-37.785105,...,478.888665,Debney Meadows Primary School,99.382328,Flemington Bridge Railway Station,482.238009,0.080110,0.036814,0.009850,0.042258,4
94,Collingwood,3066,"31 Wellington Street, Collingwood, Vic",875,2025-08-13,2,2,1,Apartment,-37.807415,...,599.751859,Collingwood English Language School,94.782438,Epworth Freemasons Hospital : Medical Centre,367.879627,0.103079,0.034064,0.007318,0.048154,5
41,North Melbourne,3051,"315 Flemington Road, North Melbourne, Vic",420,2025-08-28,1,1,1,Apartment,-37.791667,...,505.082957,St Aloysius College,181.045425,St Michaels,284.003442,0.085088,0.085632,0.005461,0.058727,6
97,Richmond,3121,"8 Garfield Street, Richmond, Vic",490,2025-08-12,1,1,1,Apartment,-37.811854,...,96.652089,Richmond West Primary School,329.969597,Darling Square,347.117656,0.007470,0.174658,0.006859,0.062996,7
90,Fitzroy,3065,"98 Nicholson Street, Fitzroy, Vic",450,2025-08-05,1,1,0,Apartment,-37.802988,...,891.443166,Academy of Mary Immaculate,85.263634,Carlton Gardens North,276.978926,0.158511,0.028374,0.005306,0.064064,8
35,Melbourne,3004,"605 St Kilda Road, Melbourne, Vic",550,2025-09-03,1,1,1,Apartment,-37.850939,...,754.071924,Victorian College For The Deaf,154.882728,Wesley College,279.409264,0.132405,0.069992,0.005360,0.069252,9
44,North Melbourne,3051,"150 Peel Street, North Melbourne, Vic",750,2025-08-08,2,1,1,Apartment,-37.803610,...,901.069199,St Joseph's,118.766861,St Mary's Anglican Church,225.242147,0.160340,0.048402,0.004161,0.070968,10


The above table shows top 10 rental listings based on their accessibility scores.

### Visualise top 10 rental listing

In [ ]:
# Average lat/lon for center
melbourne_center = [top10_listing['latitude'].mean(), top10_listing['longitude'].mean()]
top_listing_map = folium.Map(location=melbourne_center, zoom_start=13)

# Add coworking space markers
for _, row in top10_listing.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['Address'],
    ).add_to(top_listing_map)
    


# Add the layer control
folium.LayerControl().add_to(top_listing_map)

# Display the map
top_listing_map


The top 10 rental listings are visualised into a map. It can be seen that the locations are spread out in different suburbs. The majority are close to Collingwood and North Melbourne.